In [274]:
# Setting up environment
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import plotly.express as px
import numpy as np
from sklearn.metrics import classification_report
import statsmodels.formula.api as smf
from sklearn import metrics
import os
from statistics import mean
from math import isnan
from itertools import filterfalse
# Modeling Libraries
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split
from xgboost import XGBClassifier

In [275]:
os.chdir("E:\MSBA WINTER 23\BANA 277")


In [276]:
df = pd.read_csv("survey.csv")
display(df)

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17:21,26,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,...,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,2015-09-26 01:07:35,32,Male,United States,IL,No,Yes,Yes,Often,26-100,...,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,2015-11-07 12:36:58,34,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,...,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,2015-11-30 21:25:06,46,f,United States,NC,No,No,No,NaN,100-500,...,Don't know,Yes,No,No,No,No,No,No,No,NaN


In [278]:
# Dropping columns
df = df.drop(["comments","state","Timestamp"], axis=1)
df.isnull().sum()

Age                            0
Gender                         0
Country                        0
self_employed                 18
family_history                 0
treatment                      0
work_interfere               264
no_employees                   0
remote_work                    0
tech_company                   0
benefits                       0
care_options                   0
wellness_program               0
seek_help                      0
anonymity                      0
leave                          0
mental_health_consequence      0
phys_health_consequence        0
coworkers                      0
supervisor                     0
mental_health_interview        0
phys_health_interview          0
mental_vs_physical             0
obs_consequence                0
dtype: int64

In [279]:
# Cleaning data, giving default value for each data
defInt = 0
defStr = "NA"
defFl = 0.0
integer_col = ["Age"]
string_coldf = df[df.columns.difference(["Age"])]
string_col = string_coldf.columns.tolist()
float_col = []
for feature in df:
    if feature in integer_col:
        df[feature] = df[feature].fillna(defInt)
    elif feature in string_col:
        df[feature]= df[feature].fillna(defStr)
    elif feature in float_col:
        df[feature] = df[feature].fillna(defFl)
    else:
        print("No feature")
display(df)

,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,Female,United States,NA,No,Yes,Often,6-25,No,Yes,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,44,M,United States,NA,No,No,Rarely,More than 1000,No,No,...,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
2,32,Male,Canada,NA,No,No,Rarely,6-25,No,Yes,...,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,31,Male,United Kingdom,NA,Yes,Yes,Often,26-100,No,Yes,...,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,31,Male,United States,NA,No,No,Never,100-500,Yes,Yes,...,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,26,male,United Kingdom,No,No,Yes,NA,26-100,No,Yes,...,Don't know,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No
1255,32,Male,United States,No,Yes,Yes,Often,26-100,Yes,Yes,...,Yes,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No
1256,34,male,United States,No,Yes,Yes,Sometimes,More than 1000,No,Yes,...,Don't know,Somewhat difficult,Yes,Yes,No,No,No,No,No,No
1257,46,f,United States,No,No,No,NA,100-500,Yes,Yes,...,Don't know,Don't know,Yes,No,No,No,No,No,No,No


In [280]:
# Cleaning Gender Column
gender = df.Gender.str.lower()
gender = df.Gender.unique()
str_male = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
str_trans = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]           
str_female = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]
for (row, col) in df.iterrows():

    if str.lower(col.Gender) in str_male:
        df['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in str_female:
        df['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in str_trans:
        df['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)

value = ["A little about you", "p"]
df = df[df.Gender.isin(value)==False]

In [281]:
# Cleaning Age column
df.Age.fillna(df.Age.median(),inplace= True)
# People should be of working age, so change all the values
agemed= df.Age.median()
df["Age"].values[df["Age"] < 18] = agemed
df["Age"].values[df["Age"] > 100] = agemed


In [282]:
# All no data values of self employed will be assumed to be No
df.self_employed = df.self_employed.replace("NA", "No")
df.work_interefere = df.work_interfere.replace("NA", "No")
df.work_interefere.unique()

C:\Users\halan\AppData\Local\Temp\ipykernel_18072\4146232950.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.work_interefere = df.work_interfere.replace("NA", "No")


array(['Often', 'Rarely', 'Never', 'Sometimes', 'No'], dtype=object)

In [2]:
# print("family history",df.family_history.unique()) # two
# print("treatment ",df.treatment.unique() )# two
# print("work_interfere ",df.work_interfere.unique()) # 4
# print("no_employees ",df.no_employees.unique()) # 6
# print("remote_work ",df.remote_work.unique()) # two
# print("tech_company ",df.tech_company.unique()) # two
# print("benefits ",df.benefits.unique()) # three 
# print("care_options ",df.care_options.unique()) # three 
# print("wellness_program ",df.wellness_program.unique()) # three 
# print("seek_help ",df.seek_help.unique()) # three 
# print("anonymity ",df.anonymity.unique()) # three 
# print("leave ",df.leave.unique()) # 5
# print("mental_health_consequence ",df.mental_health_consequence.unique()) # three 
# print("phys_health_consequence ",df.phys_health_consequence.unique())# three 
# print("coworkers ",df.coworkers.unique())# three 
# print("supervisor ",df.supervisor.unique())# three 
# print("mental_health_interview",df.mental_health_interview.unique())# three 
# print("phys_health_interview ",df.phys_health_interview.unique())# three 
# print("family mental_vs_physical",df.mental_vs_physical.unique())# three 
# print("obs_consequence ",df.obs_consequence.unique()) # two

In [283]:
# Putting categorical values to binomial
df["self_employed"] = df.self_employed.apply(lambda x: 0 if x == "No" else 1)
df["treatment"] = df.treatment.apply(lambda x: 0 if x == "No" else 1)
df["family_history"] = df.family_history.apply(lambda x: 0 if x == "No" else 1)
df["remote_work"] = df.remote_work.apply(lambda x: 0 if x == "No" else 1)
df["tech_company"] = df.tech_company.apply(lambda x: 0 if x == "No" else 1)
df["obs_consequence"] = df.obs_consequence.apply(lambda x: 0 if x == "No" else 1)
df.head()

,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,female,United States,0,0,1,Often,6-25,0,1,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,0
1,44,male,United States,0,0,0,Rarely,More than 1000,0,0,...,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,0
2,32,male,Canada,0,0,0,Rarely,6-25,0,1,...,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,0
3,31,male,United Kingdom,0,1,1,Often,26-100,0,1,...,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,1
4,31,male,United States,0,0,0,Never,100-500,1,1,...,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,0


In [284]:
# Creating dummy variables 
dvcol = pd.get_dummies(df[["Gender","work_interfere","no_employees","benefits","care_options","wellness_program",
            "seek_help","anonymity","leave","mental_health_consequence","phys_health_consequence",
            "coworkers","supervisor","mental_health_interview","phys_health_interview",
            "mental_vs_physical"]])
display(dvcol)

,Gender_female,Gender_male,Gender_trans,work_interfere_NA,work_interfere_Never,work_interfere_Often,work_interfere_Rarely,work_interfere_Sometimes,no_employees_1-5,no_employees_100-500,...,supervisor_Yes,mental_health_interview_Maybe,mental_health_interview_No,mental_health_interview_Yes,phys_health_interview_Maybe,phys_health_interview_No,phys_health_interview_Yes,mental_vs_physical_Don't know,mental_vs_physical_No,mental_vs_physical_Yes
0,1,0,0,0,0,1,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
1,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,0
2,0,1,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,1,0,1,0
3,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
4,0,1,0,0,1,0,0,0,0,1,...,1,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,0,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,0,0
1255,0,1,0,0,0,1,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
1256,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
1257,1,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0


In [285]:
# Creating temporary column to combine values
df_final = df
df_final = pd.concat([df_final, dvcol], axis = 1)
display(df_final)


,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,...,supervisor_Yes,mental_health_interview_Maybe,mental_health_interview_No,mental_health_interview_Yes,phys_health_interview_Maybe,phys_health_interview_No,phys_health_interview_Yes,mental_vs_physical_Don't know,mental_vs_physical_No,mental_vs_physical_Yes
0,37,female,United States,0,0,1,Often,6-25,0,1,...,1,0,1,0,1,0,0,0,0,1
1,44,male,United States,0,0,0,Rarely,More than 1000,0,0,...,0,0,1,0,0,1,0,1,0,0
2,32,male,Canada,0,0,0,Rarely,6-25,0,1,...,1,0,0,1,0,0,1,0,1,0
3,31,male,United Kingdom,0,1,1,Often,26-100,0,1,...,0,1,0,0,1,0,0,0,1,0
4,31,male,United States,0,0,0,Never,100-500,1,1,...,1,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,26,male,United Kingdom,0,0,1,NA,26-100,0,1,...,0,0,1,0,0,1,0,1,0,0
1255,32,male,United States,0,1,1,Often,26-100,1,1,...,1,0,1,0,0,1,0,0,0,1
1256,34,male,United States,0,1,1,Sometimes,More than 1000,0,1,...,0,0,1,0,0,1,0,0,1,0
1257,46,female,United States,0,0,0,NA,100-500,1,1,...,0,0,1,0,0,1,0,0,1,0


In [286]:
df_final = df_final.drop(["Gender","work_interfere","no_employees","benefits","care_options","wellness_program",
            "seek_help","anonymity","leave","mental_health_consequence","phys_health_consequence",
            "coworkers","supervisor","mental_health_interview","phys_health_interview",
            "mental_vs_physical"], axis=1)

In [290]:
df_final.columns

Index(['Age', 'Country', 'self_employed', 'family_history', 'treatment',
       'remote_work', 'tech_company', 'obs_consequence', 'Gender_female',
       'Gender_male', 'Gender_trans', 'work_interfere_NA',
       'work_interfere_Never', 'work_interfere_Often', 'work_interfere_Rarely',
       'work_interfere_Sometimes', 'no_employees_1-5', 'no_employees_100-500',
       'no_employees_26-100', 'no_employees_500-1000', 'no_employees_6-25',
       'no_employees_More than 1000', 'benefits_Don't know', 'benefits_No',
       'benefits_Yes', 'care_options_No', 'care_options_Not sure',
       'care_options_Yes', 'wellness_program_Don't know',
       'wellness_program_No', 'wellness_program_Yes', 'seek_help_Don't know',
       'seek_help_No', 'seek_help_Yes', 'anonymity_Don't know', 'anonymity_No',
       'anonymity_Yes', 'leave_Don't know', 'leave_Somewhat difficult',
       'leave_Somewhat easy', 'leave_Very difficult', 'leave_Very easy',
       'mental_health_consequence_Maybe', 'mental_healt

In [1]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(df_final.corr(),annot=False)
plt.title("Correlation")
plt.show()

NameError: name 'plt' is not defined